In [1]:
import time
import math
import sympy as sp
import numpy as np
from numpy.polynomial import polynomial as p
import matplotlib.pyplot as plt
from sage.all import *
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler 

In [2]:
def gaussian_sample(sigma, shape):
    D = DiscreteGaussianDistributionIntegerSampler(sigma)
    return np.array([D() for _ in range(np.prod(shape))]).reshape(shape)

### Class thar runs and controls the protocol flow

In [3]:
class ProductArgument(object):
    
    #Parameters: security level lambda, prime p and N triples ai, bi, ci 
    #such that ai*bi = ci (flatA = [a1, a2, ..., aN])
    
    lamb = None
    p = None
    flatA = None
    flatB = None
    flatC = None
    
    
    def __init__(self, lamb, p, flatA, flatB, flatC):
        self.lamb = lamb
        self.p = p
        self.flatA = flatA
        self.flatB = flatB
        self.flatC = flatC
        
    
    def run(self):
        assert self.flatA.size == self.flatB.size == self.flatC.size
        N = self.flatA.size
        
        #Generates the commitment scheme for N values in Zp with security
        #lambda. This defines n, k, m, q ... (?)
        cs = CommitmentScheme(self.lamb, self.p, N)
        ck = cs.gen()
        
        #TO-DO: Reshape A,B,C into m matrices kxn, according to cs
        
        #Create prover
        prover = ProductArgumentProver(self.lamb, self.p, Ai, Bi, Ci, ck)
        
        #Create verifier
        verifier = ProductArgumentVerifier()
        
        #Start protocol
        start_time = time.time()
        
        prover.commit_witness()
        
        bfy = verifier.generate_challenge()
        
        prover.generate_polynomials(bfy)
        
        prover.commit_polynomial()
        
        bfx = verifier.generate_challenge()
        
        prover.evaluate_polynomials(bfx, bfy)
        
    

In [24]:
class ProductArgumentProver(object):
    lamb = None
    ck = None
    Ai = None
    Bi = None
    Ci = None
    k = None
    m = None
    n = None
    N = None
    p = None
    q = None
    np = None
    sigma1 = None
    sigma2 = None
    sigma3 = None
    sigma4 = None
    
    
    def __init__(self, lamb, p, Ai, Bi, Ci, ck):
        
        assert Ai.shape == Bi.shape == Ci.shape
        
        self.Ai = Ai
        self.Bi = Bi
        self.Ci = Ci
        self.m = Ai.shape[0]
        self.k = Ai.shape[1]
        self.n = Ai.shape[2]
        
        self.ck = ck
        self.p = ck[0]
        self.q = ck[1]
        #self.np =  1 
        
        self.sigma1 = 48*math.sqrt(self.k*self.n)*self.k*self.m*self.p**2
        self.sigma2 = 72*math.sqrt(2*self.k*self.n)*self.k*self.m*self.p
        self.sigma3 = 24*math.sqrt(2*self.k*self.n)*self.k*self.p*(1+6*self.k*self.m*self.p)
        self.sigma4 = 24*math.sqrt(2)*self.k**2*self.p*self.n*self.sigma2
    
    def commit_witness(self):
        self.Aizeros = np.vstack((self.Ai, np.zeros_like(Ai)))
        self.Bizeros = np.vstack((self.Bi, np.zeros_like(Bi)))
        self.CiCpi = Aizeros * Bizeros % self.p
        #This should be done in a suitable basis of GF(p^{2k}) (???)
        
        self.A0 = gaussian_sample(self.sigma1, (2*self.k, self.n))
        self.Bmm1 = gaussian_sample(self.sigma1, (2*self.k, self.n))
        
        self.alphai = np.random.randint(0, self.p, [self.m, self.k, self.np])
        self.betai = np.random.randint(0, self.p, [self.m, self.k, self.np])
        self.gammai = np.random.randint(0, self.p, [self.m, 2*self.k, self.np])

        self.alpha0 = gaussian_sample(self.sigma1, (2*self.k, self.np))
        self.betamm1 = gaussian_sample(self.sigma1, (2*self.k, self.np))
        
        self.bfAi = com_msg(Aizeros, np.vstack((alphai, np.zeros_like(alphai))))
        self.bfBi = com_msg(Bizeros, np.vstack((betai, np.zeros_like(betai))))
        self.bfCi = com_msg(CiCpi, gammai)
        
        self.bfA0 = com_msg(A0, alpha0)
        self.bfBmm1 = com_msg(Bmm1, betamm1)
        
        return bfAi, bfBi, bfCi, bfA0, bfBmm1
        
        
    def compute_polynomials(self, bfy):
        Mbfy = np.diag(bfy) #Matrix that emulates left product by bfy in GF(p^{2k}) (???)
        powered_Mbfy = Mbfy #On iteration i this will be Mbfy to the (i+1)-th power
        self.AX = self.A0
        
        #Compute polynomial A(X) = A0 + Aizeros[0]*y*X + Aizeros[1]*y²*X² + ... + Aizeros[m-1]*y^m*X^m
        for i in range(self.m):
            np.append(AX, powered_Mbfy @ self.Aizeros[i])
            powered_Mbfy = powered_Mbfy @ Mbfy % self.p
        
        #Compute polynomial B(X) = B_(m+1) + Bizeros[m-1]*X + Bizeros[m-2]*X² + ... + Bizeros[0]*X^m
        self.BX = np.append(self.Bmm1, np.flip(self.Bizeros, 0))    
         
        #Calculate value of C = CiCpi[0]*y + CiCpi[1]*y² + CiCpi[2]*y³ + ... + CiCpi[m-1]*y^m
        self.C = polyeval(self.CiCpi, Mbfy) % self.p #polyeval is in Utilities.ipynb
        
        self.Hl = (self.AX @ self.BX) % self.p #The (m+1) coefficient of Hl is C 
        
    
    def commit_polynomial(self):
        etal = np.random.randint(0, self.p, [2*self.m, 2*self.k, self.np])
        self.bfHl = com_msg(Hl[:self.m+1], etal[:self.m+1])
        np.append(self.bfHl, com_msg(Hl[self.m+2:], etal[self.m+1:])) 
    
    
    def evaluate_polynomials(self, bfx, bfy):
        Mbfx = np.diag(bfx) #Matrix that emulates left product by bfx in GF(p^{2k}) (???)
        Mbfy = np.diag(bfy) #Matrix that emulates left product by bfy in GF(p^{2k}) (???)
        powered_Mbfy = Mbfy #On iteration i this will be Mbfy to the (i+1)-th power
        
        self.A = self.A0
        self.alpha = self.alpha_0
        #Calculate value A = A0 + Aizeros[0]*y*x + Aizeros[1]*y²*x² + ... + Aizeros[m-1]*y^m*x^m
        for i in range(self.m):
            Mxy = powered_Mbfx @ powered_Mbfy % self.p
            self.A += Mxy @ self.Aizeros[i]
            self.alpha += Mxy @ np.vstack((alphai, np.zeros_like(alphai)))
            powered_Mbfy = powered_Mbfy @ Mbfy
            powered_Mbfx = powered_Mbfx @ Mbfx
            
        self.B = polyeval(self.BX, Mbfx)
        self.beta = self.betamm1 + polyeval(np.flip(np.vstack((self.betai, np.zeros_like(self.betai))), 0), Mbfx)
        
        #Check: A(X) = A mod p, B(X) = B mod p

In [5]:
class ProductArgumentVerifier(object):
    lamb = None
    ck = None
    calB = None
    k = None
    m = None
    n = None
    N = None
    p = None
    q = None
    np = None
    
    def __init__(self, lamb, p, m, k, n, ck):
        
        self.sigma1 = 48*math.sqrt(self.k*self.n)*self.k*self.m*self.p**2
        self.sigma2 = 72*math.sqrt(2*self.k*self.n)*self.k*self.m*self.p
        self.sigma3 = 24*math.sqrt(2*self.k*self.n)*self.k*self.p*(1+6*self.k*self.m*self.p)
        self.sigma4 = 24*math.sqrt(2)*self.k**2*self.p*self.n*self.sigma2
    
    
    # Generate challenges bf y, bf x and bf z 
    def generate_challenge(self):
        c = np.random.randint(0, self.p, 2*self.k)
        return c
    
    

In [6]:
def productArgument_test():
    
    lamb = 64
    p = 5
    N = 20
    n = 2
    k = 2
    m = 1
    
    flatA = np.random.randint(0, p, size = N)
    flatB = np.random.randint(0, p, size = N)
    flatC = flatA * flatB % p #Hadamard product
    
    print(flatA)
    print(flatB)
    print(flatC)
    
    argument = ProductArgument(lamb, p, flatA, flatB, flatC)
    argument.run()
    
#productArgument_test()

In [7]:
p = 7
n = 2
k = 2
m = 2
    
Ai = np.random.randint(0, p, size = [m, k, n])

In [8]:
A = np.random.randint(1, 5, [5,10])
x = np.random.randint(1, 5, 10)
A, x, A@x

(array([[2, 2, 3, 3, 1, 2, 4, 4, 4, 4],
        [1, 3, 1, 2, 3, 1, 1, 3, 2, 2],
        [2, 1, 4, 2, 4, 4, 3, 1, 2, 4],
        [1, 4, 3, 1, 4, 4, 4, 3, 1, 4],
        [4, 1, 1, 2, 2, 2, 2, 1, 4, 4]]),
 array([2, 2, 2, 3, 2, 1, 1, 1, 3, 1]),
 array([51, 35, 46, 45, 43]))

In [15]:
bfy = np.random.randint(1, 5, 10)
np.diag(bfy)

array([[16,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  4,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  4,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 16,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  4,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  9,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  9,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 16,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1]])

In [17]:
np.append(1, 2)

array([1, 2])

In [25]:
A = np.random.randint(1, 5, [ 2, 2])
B = np.random.randint(1, 5, [ 2, 2])
C = np.random.randint(1, 5, [ 2, 2])
A, B, C

(array([[3, 2],
        [3, 3]]), array([[1, 3],
        [4, 3]]), array([[4, 4],
        [4, 4]]))

In [26]:
A@B@C

array([[104, 104],
       [132, 132]])